# A Common Hierarchical Abstract Representation of Music: applications in historical musicology

THIS NOTEBOOK IS A WORK IN PROGRESS AND IS UNDER CONTINUAL DEVELOPMENT. 

Given a more-or-less comprehensive corpus of encoded historical lute tablatures like the augmented ‘Electronic Corpus of Lute Music’ (ECOLM) currently under development, an entirely new mode of investigation of the corpus and its relation to other repertories, such as contemporary vocal or keyboard music, could become possible. Text-based encodings of works may be directly compared using exact matching, but this does not allow recognition of transposed versions, or those for a differently-tuned instrument than a query; furthermore, duple- and triple-time versions of the same music will not, in general, be matched, nor those in which durations have been globally augmented or diminished. While ingenious indexing strategies might overcome some of these problems for a particular repertory of music (for example, the 16th-century lute fantasy and related genres), they are unlikely to permit extra-corpus searches for parallel motives or passages within contemporary vocal music, which have long been recognised as crucial in the evolution of western lute music.  
  
We present CHARM, an abstract, hierarchical music knowledge representation system that admits federation of data sources in multiple formats by means of semantic-level annotation and query specification. From the user’s perspective, search and discovery operations are specified in terms of musical terminology applied to “constituents” (groups of musical objects – e.g., notes, phrases, motives, etc.), independently of the data source and format. CHARM is equipped with data interfaces for MIDI and TabCode (other formats in progress), and it can represent audio recordings, allowing detailed association of individual notes with specific spectral content, permitting the annotation of richly detailed analyses of sources and their direct, detailed connection with performance recordings.

In this notebook we demonstrate how CHARM can be used to construct music knowledge bases from encodings of lute tablature. Using CHARM to construct knowledge bases requires the implementation of interfaces for concrete data sources and engineering the coordination of these interfaces. Here we use the Julia programming language, which has an expressive type system and method overloading, the combination of which is exploited to implement interfaces with type-based dynamic dispatch.

The code below contains 1) the CHARM definition in Julia 2) a CHARM interface for tabcode 3) an example knowledge base for a tabcode file 4) an augmentation of this representation with a representation of the topline 5) how to knowledge bases can be queried using the CHARM query language.

## CHARM

In [3]:
module CHARM

export option, none, fnd, geta, getp, pts, dom, cts

struct None end
none = None()
option{A} = Union{A,None}

# Abstract Types

abstract type Id end
abstract type Constituent end
abstract type Hierarchy end
abstract type Attribute{N,T} end
abstract type Property{N,T} end

# Registered Attributes and Properties 

__attributes__(::Val{N}) where N = error("No attribute named $N.")
__attributes__(N::Symbol) = __attributes__(Val{N}())
__properties__(::Val{N}) where N = error("No property named $N.")
__properties__(N::Symbol) = __properties__(Val{N}())

# Interface Operations

fnd(x::Id,h::Hierarchy) = none
fnd(x::Id,m::Module) = fnd(x,m.__data__)

geta(::Attribute,c::Constituent) = none
geta(N::Symbol,c) = geta(__attributes__(Val{N}()),c)

getp(::Property,c::Constituent) = none
getp(N::Symbol,c) = getp(__properties__(Val{N}()),c)

pts(c::Constituent) = Id[]

dom(h::Hierarchy) = Id[]
dom(m::Module) = dom(m.__data__)

#sequence(ids,kb) = [fnd(x,kb) for x in ids]
#cts(kb) = sequence(dom(kb),kb)

cts(kb) = Pair{Id,Constituent}[x=>fnd(x,kb) for x in dom(kb)]

sequence(ids,kb) = begin
    isempty(ids) && return Constituent[]
    c = fnd(ids[1],kb)
    c isa None && return none
    r = sequence(ids[2:end],kb)
    r isa None && return none
    return Constituent[c,r...]
end

# PROPERTIES

export DESCRIPTION, TYPE

struct DESCRIPTION <: Property{:DESCRIPTION,String} end
__properties__(::Val{:DESCRIPTION}) = DESCRIPTION()

struct TYPE <: Property{:TYPE,String} end
__properties__(::Val{:TYPE}) = TYPE()

# ATTRIBUTES

export PITCH, DUR

struct PITCH <: Attribute{:PITCH,Int} end
__attributes__(::Val{:PITCH}) = PITCH()

struct DUR <: Attribute{:DUR,String} end
__attributes__(::Val{:DUR}) = DUR()

end

using Main.CHARM

### CHARM Query Language

In [5]:
PAIR(X,Y) = Pair{x,y} where {x<:X,y<:Y}
LST(X) = Vector{x} where {x<:X}
OPT(X) = Union{X,CHARM.None}
ID = CHARM.Id
C = CHARM.Constituent
CON = PAIR(ID,C)
G = LST(CON)

domain(cs::G) = first.(cs)

mem(x::ID,cs::G)::Bool = x in domain(cs)
mem(xs::LST(ID),cs::G) = all([mem(x,cs) for x in xs])

position(x::ID,cs::G) = findfirst(==(x),domain(cs))

lookup(x::ID,cs::G) = mem(x,cs) ? cs[position(x,cs)] : none

lookup(xs::LST(ID),cs::G)::option{CON} = begin
    res = Pair{CHARM.Id,CHARM.Constituent}[]
    for x in xs
        c = lookup(x,cs)
        c == none && return none
        union!(res,[c])
    end
    return res
end

pop(x::ID,cs::G) = begin 
    !mem(x,cs) && return CON[]
    return cs[position(x,cs):end]
end

parts(x::ID,cs::G) = begin 
    !mem(x,cs) && return none
    c = lookup(x,cs)
    return lookup(pts(c[2]),cs)
end
parts(c::CON,cs::G) = lookup(pts(c[2]),cs)

subparts(x::ID,cs::G) = begin
    isempty(cs) && return CON[]
    c = lookup(x,cs)
    c == none && return CON[]
    res = CON[c]
    ps = pts(c[2])
    for p in ps
        union!(res,subparts(p,pop(p,cs)))
    end
    return res
end

subparts(xs::LST(ID),cs::G) = begin
    res = CON[]
    [union!(res,subparts(x,cs)) for x in xs]
    return [c for c in cs if c in res]
end

superparts(x::ID,cs::G) = begin
    isempty(cs) && return CON[]
    c = lookup(x,cs)
    c == none && return CON[]
    res = CON[c]
    for c2 in reverse(cs[1:position(x,cs)])
        if x in pts(c2[2])
            push!(res,c2)
            union!(res,superparts(c2[1],cs))
        end
    end
    return reverse(res)
end

superparts(xs::LST(ID),cs::G) = begin 
    res = CON[]
    [union!(res,superparts(x,cs)) for x in xs]
    return [c for c in cs if c in res]
end

haspart(x::ID,y::ID,cs::G) = begin
    !(mem(x,cs)) && return false
    return y in first.(parts(x,cs))
end

ispartof(x::ID,y::ID,cs::G) = haspart(y,x,cs)

hassubpart(x,y,cs) = begin
    y in first.(subparts(x,cs))
end

issubpartof(x::ID,y::ID,cs::G) = hassubpart(y,x,cs)

isdag(cs::G) = begin 
    isempty(cs) && return true
    x,c = cs[1]
    t = cs[2:end]
    ps = pts(c)
    mem(x,t) && (print("!!!"); return false)
    !mem(ps,t) && (print("!!!$x"); return false)
    !isdag(t) && (print("!!!"); return false)
    return true
end
isleaf(x::ID,cs::G) = begin 
    c = lookup(x,cs) 
    return c != none && isempty(pts(c[2]))
end
istree(cs::G) = begin
    isempty(cs) && return true
    x,c = cs[1]
    t = cs[2:end]
    sps = [subparts(p[1],cs) for p in parts(x,cs)]
    for p in parts(x)
        return isleaf(p,cs) || istree(p,cs)
    end
end

PTY{N,T} = CHARM.Property{N,t} where {t<:T}
ATT{N,T} = CHARM.Attribute{N,t} where {t<:T}

# READ(A) : G -> option A x G 

abstract type READ{A} end

(R::READ{A})(cs::G) where A = R.op(cs)::option{A}

struct Read{A} <: READ{A}
    op::Function
    Read(A,f) = new{A}(f)
    Read(A) = new{A}(cs->none)
end

abstract type KLEISLI{A,B} end

function (K::KLEISLI{A,B})(a) where {A,B}
    a==none && return Read(B)
    Read(B,K.op(a))
end

function(K::KLEISLI{A,B})(a::A,cs::G) where {A,B}
    K(a)(cs)
end

struct BIND{A,B} <: READ{B}
    op
    BIND(R::READ{A},F::KLEISLI{A,B}) where {A,B} = new{A,B}(cs->F(R(cs),cs))
end

struct COMP{A,B} <: KLEISLI{A,B}
    op
    COMP(F::KLEISLI{A,B},G::KLEISLI{B,C}) where {A,B,C} = new{A,C}(x->cs->G(F(x,cs),cs))
end

COMP(f::KLEISLI,gs::KLEISLI...) = isempty(gs) ? f : COMP(f,COMP(gs...))

QUERY(R::READ,KS::KLEISLI...) = BIND(R,COMP(KS...))

struct PTS <: KLEISLI{CON,LST(CON)}
    op
    PTS() = new(c->cs->parts(c,cs))
end

struct GETA{N,T} <: KLEISLI{CON,T}
    op
    GETA(a::ATT{N,T}) where {N,T} = new{N,T}(c->cs->geta(a,c[2]))
    GETA(N::Symbol) = GETA(CHARM.__attributes__(N))
end

struct GETP{N,T} <: KLEISLI{CON,T}
    op
    GETP(a::PTY{N,T}) where {N,T} = new{N,T}(c->cs->getp(a,c[2]))
    GETP(N::Symbol) = GETP(CHARM.__properties__(N))
end

struct HASA <: KLEISLI{CON,Bool}
    op
    HASA(N::Symbol,v) = new(c->cs->geta(N,c[2])==v)
    HASA(N::Symbol) = new(c->cs->geta(N,c[2])!=none)
end

struct HASP <: KLEISLI{CON,Bool}
    op
    HASP(N::Symbol,v) = new(c->cs->getp(N,c[2])==v)
    HASP(N::Symbol) = new(c->cs->getp(N,c[2])!=none)
end

struct FIRSTPART <: KLEISLI{CON,Bool}
    op
    FIRSTPART(T::KLEISLI) = new(c->cs->(ps = parts(c,cs); isempty(ps) ? false : T(first(ps))(cs)))
    FIRSTPART(x::ID) = FIRSTPART(c->cs->c[1]==x)
end

struct LASTPART <: KLEISLI{CON,Bool}
    op
    LASTPART(T) = new(c->cs->(ps = parts(c,cs); isempty(ps) ? false : T(last(ps))(cs)))
    LASTPART(x::ID) = LASTPART(c->cs->c[1]==x)
end

struct ALLPARTS <: KLEISLI{CON,Bool}
    op
    ALLPARTS(T) = new(c->cs->all([T(p,cs) for p in parts(c,cs)])) 
end

struct SOMEPART <: KLEISLI{CON,Bool}
    op
    SOMEPART(T) = new(c->cs->any([T(p,cs) for p in parts(c,cs)]))
end

struct AND <: KLEISLI{CON,Bool}
    op
    AND(f::KLEISLI,g::KLEISLI) = new(a->cs->f(a,cs)&&g(a,cs))
    AND(f::KLEISLI,gs::KLEISLI...) = isempty(gs) ? f : AND(AND(f,gs[1]),gs[2:end]...)
end

struct SELECT <: READ{G}
    op
    SELECT(T::KLEISLI{CON,Bool}) = new(cs->[c for c in cs if T(c)(cs)])
    SELECT(x) = new(cs->[lookup(x,cs)])
end

opmap(B,f,l) = begin
    isempty(l) && return B[]
    h = f(l[1])
    t = opmap(B,f,l[2:end])
    h == none && return none
    t == none && return none
    return B[h,t...]
end

struct MAP{A,B} <: KLEISLI{LST(A),LST(B)}
    op
    MAP(f::KLEISLI{A,B}) where {A,B} = new{A,B}(l->cs->opmap(B,x->f(x)(cs),l))
end

struct RET{A} <: READ{A}
    op
    RET(v::A) where A = new{A}(_->v)
end

struct SUBCS <: KLEISLI{CON,G}
    op
    SUBCS() = new(c->cs->subparts(c,cs))
end

struct PROJ <: KLEISLI{CON,G}
    op
    PROJ(T::KLEISLI{CON,Bool}) = new(c->cs->CON[x for x in subparts(c[1],cs) if T(x,cs)])
end

struct SUM <: KLEISLI{LST(Float64),Float64}
    op
    SUM() = new(l->cs->sum(l))
end

struct AVG <: KLEISLI{LST(Float64),Float64}
    op
    AVG() = new(l->cs->sum(l)/length(l))
end

## Tabcode Example

```
{<rules>
<tuning_named>renaissance</tuning_named>
<pitch>67</pitch>
 <rhythm-font>varietie</rhythm-font>
<bass_tuning>(-2)</bass_tuning>
</rules>}
M(C/)
Qa1a2b3c4a6
E.c4
Td2
b2
Ea2b3c4
d3
Sd1a2
b2
a2
d3
|
Sa1b3d5
d3
b3
a3
c4d5
a3
b3
d3
e2a3c5
c2
a2
c2
e2a3c5
a1
c1
e2
|
```

## Parse to JSON with Parsetab

Install parsetab: https://github.com/jamieforth/parsetab

Convert tabcode `input.tc` to json.

```
parsetab tc2json input.tc
```

```json
[{"code":"{<rules>\n<tuning_named>renaissance</tuning_named>\n<pitch>67</pitch>\n <rhythm-font>varietie</rhythm-font>\n<bass_tuning>(-2)</bass_tuning>\n</rules>}","fType":"Ruleset","rules":{"notation":"French","pitch":67,"tuning_named":"renaissance","rhythm-font":"varietie","bass_tuning":"(-2)"}},{"code":"M(C/)","tType":"Metre","components":["C/"],"vertical":false},{"tType":"Chord","duration":{"code":"Q","tType":"Duration"},"mainCourses":[{"tType":"TabNote","pitch":{"code":"a1","tType":"Pitch","fret":"a","course":1}},{"tType":"TabNote","pitch":{"code":"a2","tType":"Pitch","fret":"a","course":2}},{"tType":"TabNote","pitch":{"code":"b3","tType":"Pitch","fret":"b","course":3}},{"tType":"TabNote","pitch":{"code":"c4","tType":"Pitch","fret":"c","course":4}},{"tType":"TabNote","pitch":{"code":"a6","tType":"Pitch","fret":"a","course":6}}],"bassCourses":[]},{"tType":"Chord","duration":{"code":"E.","tType":"Duration"},"mainCourses":[{"tType":"TabNote","pitch":{"code":"c4","tType":"Pitch","fret":"c","course":4}}],"bassCourses":[]},{"tType":"Chord","duration":{"code":"T","tType":"Duration"},"mainCourses":[{"tType":"TabNote","pitch":{"code":"d2","tType":"Pitch","fret":"d","course":2}}],"bassCourses":[]},{"tType":"Chord","duration":{"code":"T","tType":"Duration"},"mainCourses":[{"tType":"TabNote","pitch":{"code":"b2","tType":"Pitch","fret":"b","course":2}}],"bassCourses":[]},{"tType":"Chord","duration":{"code":"E","tType":"Duration"},"mainCourses":[{"tType":"TabNote","pitch":{"code":"a2","tType":"Pitch","fret":"a","course":2}},{"tType":"TabNote","pitch":{"code":"b3","tType":"Pitch","fret":"b","course":3}},{"tType":"TabNote","pitch":{"code":"c4","tType":"Pitch","fret":"c","course":4}}],"bassCourses":[]},{"tType":"Chord","duration":{"code":"E","tType":"Duration"},"mainCourses":[{"tType":"TabNote","pitch":{"code":"d3","tType":"Pitch","fret":"d","course":3}}],"bassCourses":[]},{"tType":"Chord","duration":{"code":"S","tType":"Duration"},"mainCourses":[{"tType":"TabNote","pitch":{"code":"d1","tType":"Pitch","fret":"d","course":1}},{"tType":"TabNote","pitch":{"code":"a2","tType":"Pitch","fret":"a","course":2}}],"bassCourses":[]},{"tType":"Chord","duration":{"code":"S","tType":"Duration"},"mainCourses":[{"tType":"TabNote","pitch":{"code":"b2","tType":"Pitch","fret":"b","course":2}}],"bassCourses":[]},{"tType":"Chord","duration":{"code":"S","tType":"Duration"},"mainCourses":[{"tType":"TabNote","pitch":{"code":"a2","tType":"Pitch","fret":"a","course":2}}],"bassCourses":[]},{"tType":"Chord","duration":{"code":"S","tType":"Duration"},"mainCourses":[{"tType":"TabNote","pitch":{"code":"d3","tType":"Pitch","fret":"d","course":3}}],"bassCourses":[]},
```

## CHARM Interface for Tabcode

In [12]:
Base.get([1,2],3,false)

false

In [13]:
module Tabcode

using Main.CHARM

abstract type Id <: CHARM.Id end

abstract type Constituent <: CHARM.Constituent end

abstract type Hierarchy <: CHARM.Hierarchy end

abstract type Attribute{N,T} <: CHARM.Attribute{N,T} end

struct Note <: Constituent
    course
    fret
    dur
    chord
    bar
    system
    page
end

struct Chord <: Constituent
    dur
    bar
    system
    page
    notes
end

struct Bar <: Constituent
    system
    page
    chords
end

struct System <: Constituent
    page
    bars
end

struct Page <: Constituent
    systems
end

struct File <: Hierarchy
    notes
    chords
    bars
    systems
    pages
end

struct FRET <: Attribute{:FRET,String} end
CHARM.__attributes__(::Val{:FRET}) = FRET()

struct COURSE <: Attribute{:COURSE,Int} end
CHARM.__attributes__(::Val{:COURSE}) = COURSE()

struct BAR <: Attribute{:BAR,String} end
CHARM.__attributes__(::Val{:BAR}) = BAR()

durnos = Dict('B'=>8.0,'W'=>4.0,'H'=>2.0,'Q'=>1.0,'E'=>0.5,'S'=>0.25,'T'=>0.125,'Y'=>0.0625,'Z'=>0.03125)

getdurno(s) = begin
    x = durnos[s[1]]
    length(s)==1 && return x
    return x + (x/2)
end

struct NDUR <: CHARM.Attribute{:NDUR,Float64} end
CHARM.__attributes__(::Val{:NDUR}) = NDUR()

CHARM.getp(::TYPE,::Note) = "Note"
CHARM.getp(::TYPE,::Chord) = "Chord"
CHARM.getp(::TYPE,::Bar) = "Bar"
CHARM.getp(::TYPE,::System) = "System"
CHARM.getp(::TYPE,::Page) = "Page"

CHARM.geta(::FRET,x::Note) = x.fret
CHARM.geta(::COURSE,x::Note) = x.course
CHARM.geta(::DUR,x::Note) = x.dur
CHARM.geta(::DUR,x::Chord) = x.dur
CHARM.geta(::NDUR,x::Note) = getdurno(geta(:DUR,x))
CHARM.geta(::BAR,x::Chord) = x.bar

using JSON

function parse(fn)

    hasttype(s,g) = haskey(g,"tType") && g["tType"] == s

    gs = JSON.parsefile(fn)
    
    notes, chords, bars, systems, pages = [], [], [], [], []
    n, c, b, s, p = 1, 1, 1, 1, 1
    ns, cs, bs, ss, ps = [], [], [], [], []
    
    for g in gs
        if hasttype("Chord",g)
            push!(cs,c)        
            
            dur = g["duration"]["code"]
    
            for x in g["mainCourses"]
                push!(ns,n)
                
                course = x["pitch"]["course"]
                fret = x["pitch"]["fret"]
                
                note = Note(course,fret,dur,c,b,s,p)
                push!(notes,note)
                n+=1
            end
    
            chord = Chord(dur,b,s,p,ns)
            push!(chords,chord)
            
            ns = []
            c+=1
        end
        if hasttype("Barline",g)
            push!(bs,b)
            bar = Bar(s,p,cs)
            push!(bars,bar)
            cs = []
            b+=1
        end
        if hasttype("SystemBreak",g)
            push!(ss,s)
            system = System(p,bs)
            push!(systems,system)
            bs = []
            s+=1
        end
        if hasttype("PageBreak",g)
            push!(ps,p)
            page = Page(ss)
            push!(pages,page)
            ss = []
            p+=1
        end
    end
    system = System(p,bs)
    push!(systems,system)
    push!(ss,s)
    page = Page(s)
    push!(pages,page)

    return File(notes, chords, bars, systems, pages)

end

end

Main.Tabcode

In [14]:
using Main.Tabcode

## Midi note number interface

In [16]:
module RenaissancePitch

using Main.CHARM
using Main.Tabcode

courses = [67,62,57,53,48,42]
frets = Dict("a"=>2,"b"=>4,"c"=>5,"d"=>7,"e"=>9,"f"=>10,"h"=>11)

getmidinote(cs,fs,c,f) = cs[c]+fs[f]

CHARM.geta(::PITCH,x::Tabcode.Note) = begin 
    c = geta(:COURSE,x)
    f = geta(:FRET,x)
    return getmidinote(courses,frets,c,f)
end

end

Main.RenaissancePitch

## Example Representation

In [18]:
module Example

using Main.CHARM
using Main.Tabcode
using Main.RenaissancePitch

abstract type Id <: CHARM.Id end

struct Note{n} <: Id end
struct Chord{n} <: Id end
struct Bar{n} <: Id end
struct System{n} <: Id end
struct Page{n} <: Id end

CHARM.pts(x::Tabcode.Note) = CHARM.Id[]
CHARM.pts(x::Tabcode.Chord) = CHARM.Id[Note{x}() for x in x.notes]
CHARM.pts(x::Tabcode.Bar) = CHARM.Id[Chord{x}() for x in x.chords]
CHARM.pts(x::Tabcode.System) = CHARM.Id[Bar{x}() for x in x.bars]
CHARM.pts(x::Tabcode.Page) = CHARM.Id[System{x}() for x in x.systems]

CHARM.fnd(x::Note{n},f::Tabcode.File) where n = get(f.notes,n,none)
CHARM.fnd(x::Chord{n},f::Tabcode.File) where n = get(f.chords,n,none) 
CHARM.fnd(x::Bar{n},f::Tabcode.File) where n = get(f.bars,n,none)
CHARM.fnd(x::System{n},f::Tabcode.File) where n = get(f.systems,n,none)
CHARM.fnd(x::Page{n},f::Tabcode.File) where n = get(f.pages,n,none)

CHARM.dom(f::Tabcode.File) = vcat([Page{p}() for p in 1:length(f.pages)],[System{s}() for s in 1:length(f.systems)],[Bar{b}() for b in 1:length(f.bars)],[Chord{c}() for c in 1:length(f.chords)],[Note{n}() for n in 1:length(f.notes)])

__data__ = Tabcode.parse("output.json")

end

Main.Example

In [19]:
using Main.Example

In [20]:
dom(Example)

2098-element Vector{Main.Example.Id}:
 Main.Example.Page{1}()
 Main.Example.Page{2}()
 Main.Example.Page{3}()
 Main.Example.System{1}()
 Main.Example.System{2}()
 Main.Example.System{3}()
 Main.Example.System{4}()
 Main.Example.System{5}()
 Main.Example.System{6}()
 Main.Example.System{7}()
 Main.Example.System{8}()
 Main.Example.System{9}()
 Main.Example.System{10}()
 ⋮
 Main.Example.Note{1199}()
 Main.Example.Note{1200}()
 Main.Example.Note{1201}()
 Main.Example.Note{1202}()
 Main.Example.Note{1203}()
 Main.Example.Note{1204}()
 Main.Example.Note{1205}()
 Main.Example.Note{1206}()
 Main.Example.Note{1207}()
 Main.Example.Note{1208}()
 Main.Example.Note{1209}()
 Main.Example.Note{1210}()

In [21]:
fnd(Example.System{10}(),Example) |> pts

3-element Vector{Main.CHARM.Id}:
 Main.Example.Bar{26}()
 Main.Example.Bar{27}()
 Main.Example.Bar{28}()

In [22]:
fnd(Example.Note{1207}(),Example) |> (n)->geta(:PITCH,n)

62

## Representing the topline

In [24]:
module Topline

using Main.CHARM
using Main.Tabcode
using Main.Example

topline = [Example.Note{c.notes[1]}() for c in Example.__data__.chords]

export Topline

struct Id <: CHARM.Id end
struct TL <: CHARM.Constituent end
struct Hierarchy <: CHARM.Hierarchy end

__data__ = Hierarchy()

CHARM.fnd(::Id,::Hierarchy) = TL()
CHARM.pts(::TL) = topline

CHARM.dom(::Hierarchy) = CHARM.Id[Id()]

end

Main.Topline

In [25]:
using Main.Topline

## Integrating Hierarchies

In [27]:
module KB

using Main.CHARM
using Main.Tabcode
using Main.Example
using Main.Topline

struct Data <: CHARM.Hierarchy end
__data__ = Data()

CHARM.fnd(x::Example.Id,::Data) = fnd(x,Example)
CHARM.fnd(x::Topline.Id,::Data) = fnd(x,Topline)
CHARM.dom(::Data) = [dom(Topline)...,dom(Example)...]

end

using Main.KB

In [28]:
cs = cts(KB);

In [29]:
fnd(Topline.Id(),KB) |> pts 

813-element Vector{Main.Example.Note}:
 Main.Example.Note{1}()
 Main.Example.Note{6}()
 Main.Example.Note{7}()
 Main.Example.Note{8}()
 Main.Example.Note{9}()
 Main.Example.Note{12}()
 Main.Example.Note{13}()
 Main.Example.Note{15}()
 Main.Example.Note{16}()
 Main.Example.Note{17}()
 Main.Example.Note{18}()
 Main.Example.Note{21}()
 Main.Example.Note{22}()
 ⋮
 Main.Example.Note{1196}()
 Main.Example.Note{1197}()
 Main.Example.Note{1198}()
 Main.Example.Note{1199}()
 Main.Example.Note{1200}()
 Main.Example.Note{1201}()
 Main.Example.Note{1202}()
 Main.Example.Note{1203}()
 Main.Example.Note{1204}()
 Main.Example.Note{1205}()
 Main.Example.Note{1209}()
 Main.Example.Note{1210}()

## Querying the Knowledge Base

In [31]:
cs |> SELECT(AND(HASP(:TYPE,"Chord"),HASA(:BAR,10)))

26-element Vector{Pair{Main.CHARM.Id, Main.CHARM.Constituent}}:
 Main.Example.Chord{127}() => Main.Tabcode.Chord("S", 10, 4, 1, Any[195, 196])
 Main.Example.Chord{128}() => Main.Tabcode.Chord("T", 10, 4, 1, Any[197])
 Main.Example.Chord{129}() => Main.Tabcode.Chord("T", 10, 4, 1, Any[198])
 Main.Example.Chord{130}() => Main.Tabcode.Chord("T", 10, 4, 1, Any[199])
 Main.Example.Chord{131}() => Main.Tabcode.Chord("T", 10, 4, 1, Any[200])
 Main.Example.Chord{132}() => Main.Tabcode.Chord("T", 10, 4, 1, Any[201])
 Main.Example.Chord{133}() => Main.Tabcode.Chord("T", 10, 4, 1, Any[202])
 Main.Example.Chord{134}() => Main.Tabcode.Chord("S", 10, 4, 1, Any[203])
 Main.Example.Chord{135}() => Main.Tabcode.Chord("S", 10, 4, 1, Any[204])
 Main.Example.Chord{136}() => Main.Tabcode.Chord("S", 10, 4, 1, Any[205])
 Main.Example.Chord{137}() => Main.Tabcode.Chord("S", 10, 4, 1, Any[206])
 Main.Example.Chord{138}() => Main.Tabcode.Chord("S", 10, 4, 1, Any[207, 208])
 Main.Example.Chord{139}() => Main.Tab